# Homework - Monte Carlo Simulation

강의에서 상관관계가 있는 난수를 제작해 동시에 두 개의 주가 시나리오를 제작해 보았습니다.<br>
이 과제에서는 동시에 두 개의 시나리오를 제작하지 않고, 하나의 시나리오를 제작하고 그를 이용하여 다른 시나리오를 완성합니다. <br>

## Heston Model

블랙 숄즈 모형의 한계를 극복하는 방법 중 하나인 확률적 변동성 모형의 한 종류인 Heston 모형은 주가가 다음과 같은 확률과정을 따른다고 가정합니다.
$$dS = rSdt + {\sqrt {v_t}\sqrt {dt}}S dW_1$$
$$dv_t = \kappa (\theta-v_t)dt + \sigma_t \sqrt {v_t}S dW_2$$
$$dW_1 dW_2 = \rho dt$$
$$\sigma_t = \sigma\sqrt{dt}$$

상관관계가 있는 두 난수를 이용한 몬테카를로 시뮬레이션을 실행하여 다음 조건 하에서 콜옵션의 가격을 구하시오.
$$S_0 = 100$$
$$v_0 = 0.3$$
$$\kappa = 0.1$$
$$K = 100$$
$$\theta = 0.2$$
$$\rho = 0.5$$
$$\sigma = 0.1$$
$$r = 0.02$$
$$T = 1$$
Hint : 먼저 $v_t$의 시나리오를 완성시키고, $v_t$ 시나리오를 이용해 $S_t$의 시나리오를 만들 수 있습니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy
from scipy.stats import norm

In [2]:
s0 = 100
v0 = 0.3
kappa = 0.1
K = 100
theta = 0.2
rho = 0.5
sigma = 0.1
r = 0.02
T = 1
days = 250
nsimulation = 10000

In [3]:
Normal1 = np.random.normal(size=(T*days, nsimulation))
Normal2 = np.random.normal(size=(T*days, nsimulation))
nor1 = Normal1.reshape(1,T*days*nsimulation)
nor2 = Normal2.reshape(1,T*days*nsimulation)
nor2 = rho*nor1 + np.sqrt(1-rho**2)*nor2
Normal1 = nor1.reshape(T*days,nsimulation)
Normal2 = nor2.reshape(T*days,nsimulation)

In [4]:
dt = 1/250
pathv = np.zeros((T*days,nsimulation))
pathv[0] = v0
for i in range(1,T*days):
    thetav = theta*np.ones(10000,)
    drift = kappa*(thetav-pathv[i-1])
    random = np.sqrt(dt)*sigma*Normal2[i-1]*np.sqrt(pathv[i-1])
    pathv[i] = pathv[i-1]+drift*dt+random

In [5]:
pathS = np.zeros((T*days+1,nsimulation))
pathS[0] = s0
for i in range(1,T*days+1):
    rS = r*dt*pathS[i-1]
    vS = np.sqrt(pathv[i-1])*pathS[i-1]*Normal1[i-1]*np.sqrt(dt)
    pathS[i] = pathS[i-1]+rS+vS

In [6]:
np.mean(np.maximum(pathS[-1]-100,0))*np.exp(-r*T)

22.489971064620924

heston 모형의 closed form solution과 비교했을 때 큰 차이가 나지 않는 것을 알 수 있습니다

In [7]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):

    # constants
    a = kappa*theta
    b = kappa+lambd

    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j

    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )

    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)

    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)

    return exp1*term2*exp2

def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    return numerator/denominator

def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

    P, umax, N = 0, 100, 10000
    dphi=umax/N #dphi is width

    for i in range(1,N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2 # midpoint to calculate height
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)

        P += dphi * numerator/denominator

    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [8]:
heston_price_rec(100, 100, 0.3, 0.1, 0.2, 0.1, 0.5, 0.2 ,1, 0.03)

22.596944892633704

위 heston_price_rec 함수에서 lambd는 variance risk premium을 의미하는데, 이 과제에서는 임의로 지정했지만 원래는 마켓 데이터와 맞는 값을 찾아 사용합니다.